In [54]:
from pathlib import Path

import yaml
import openai
import pandas as pd

In [2]:
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

True

In [33]:
client = openai.OpenAI()

In [ ]:
p = Path('recordings')

In [12]:
throat_small = sorted(list(p.glob('External_Microphone/*.wav')))

In [13]:
len(throat_small)

100

In [14]:
throat_small[0]

PosixPath('recordings/External_Microphone/20251021232813.wav')

In [19]:
throat_small_enhanced = sorted(list(p.glob('External_Microphone_Enhanced/*.wav')))

In [20]:
len(throat_small_enhanced)

100

In [17]:
throat_large = sorted(list(p.glob('GHW_USB_AUDIO/*.wav')))

In [18]:
len(throat_large)

100

In [21]:
throat_large_enhanced = sorted(list(p.glob('GHW_USB_AUDIO_Enhanced/*.wav')))

In [24]:
reference_mic = sorted(list(p.glob('Marantz_Umpire_Mic/*.wav')))

In [25]:
len(reference_mic)

100

In [27]:
with Path('basic5000.yaml').open('r') as f:
    basic5000 = yaml.safe_load(f)

In [32]:
basic5000['BASIC5000_0001']['text_level0']

'水をマレーシアから買わなくてはならないのです。'

In [41]:
correct_texts = []
for i in range(1, 101):
    key = f"BASIC5000_{i:04d}"
    correct_texts.append(basic5000[key]['text_level0'])

In [47]:
def transcribe_sync_from_file(file_path):
    with open(file_path, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            language="ja",
            temperature=0.2,
        )

    return transcript.text

In [48]:
correct_texts[0]

'水をマレーシアから買わなくてはならないのです。'

In [49]:
transcribe_sync_from_file(reference_mic[0])

'水をマレーシアから買わなくてはならないのです。'

In [50]:
transcribe_sync_from_file(throat_small[0])

'いつもマレーシアから買わなくてはならないのです。'

In [51]:
transcribe_sync_from_file(throat_small_enhanced[0])

'いつも、毎日、やからかは、乗ってはならないのです。'

In [52]:
transcribe_sync_from_file(throat_large[0])

'別でマレーシアからカンパウンドのような音です。'

In [53]:
transcribe_sync_from_file(throat_large_enhanced[0])

'肉を焼きしながら食べます。'

In [55]:
df = pd.DataFrame({
    'correct_text': correct_texts,
    'reference_mic': reference_mic,
    'throat_small': throat_small,
    'throat_small_enhanced': throat_small_enhanced,
    'throat_large': throat_large,
    'throat_large_enhanced': throat_large_enhanced,
})

In [58]:
df['reference_mic_transcript'] = df['reference_mic'].apply(transcribe_sync_from_file)
df['throat_small_transcript'] = df['throat_small'].apply(transcribe_sync_from_file)
df['throat_small_enhanced_transcript'] = df['throat_small_enhanced'].apply(transcribe_sync_from_file)
df['throat_large_transcript'] = df['throat_large'].apply(transcribe_sync_from_file)
df['throat_large_enhanced_transcript'] = df['throat_large_enhanced'].apply(transcribe_sync_from_file)

In [60]:
df.to_pickle('throat2air_sklearn_result.pkl')

In [61]:
df.head()

,correct_text,reference_mic,throat_small,throat_small_enhanced,throat_large,throat_large_enhanced,reference_mic_transcript,throat_small_transcript,throat_small_enhanced_transcript,throat_large_transcript,throat_large_enhanced_transcript
0,水をマレーシアから買わなくてはならないのです。,recordings/Marantz_Umpire_Mic/20251021232813.wav,recordings/External_Microphone/20251021232813.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021232813.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212328...,水をマレーシアから買わなくてはならないのです。,いつもマレーシアから買わなくてはならないのです。,いつも、毎日、やかな顔のってはならないのです。,別でマレーシアからカナダの方への呼びかけです。,肉を骨にしながら食べます。
1,木曜日、停戦会談は、何の進展もないまま終了しました。,recordings/Marantz_Umpire_Mic/20251021233140.wav,recordings/External_Microphone/20251021233140.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021233140.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212331...,木曜日、停線階段は何の進展もないまま終了しました。,木曜日、停線階段は何の進展もないまま終了しました。,木曜日、停線改段は何の進展もないまま終了しました。,木曜日、定山会談はこの進展のみを終了しました。,木曜日、テイスラル会談はパワーシンティアの 今後の次期をお知らせします。
2,上院議員は私がデータをゆがめたと告発した。,recordings/Marantz_Umpire_Mic/20251021233208.wav,recordings/External_Microphone/20251021233208.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021233208.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212332...,上院議員は私がデータを歪めたと告発した。,上院議員は私がデータを歪めたと告発した。,教員入院は私がデータを歪めたと告発した。,JNBはあたしのデータを入れ替えたと告白した。,教員の指輪には同じな結果を入れ込めたと報告しました。
3,１週間して、そのニュースは本当になった。,recordings/Marantz_Umpire_Mic/20251021233250.wav,recordings/External_Microphone/20251021233250.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021233250.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212332...,一週間して、そのニュースは本当になった。,一週間してそのニュースは本当になった,一週間してそのニュースは本当になった。,一生懸命して、そのネオイスは本当にいなかった。,一周して、この状況は反抗になった。
4,血圧は、健康のパロメーターとして重要である。,recordings/Marantz_Umpire_Mic/20251021233319.wav,recordings/External_Microphone/20251021233319.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021233319.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212333...,血圧は健康のバロメータとして重要である。,血圧は健康のバロメータとして重要である。,血圧は健康のバロンイーターとして重要である。,お客さんさん、健康のために隠して重要です。,


In [68]:
# pip install openai python-dotenv tqdm pandas tenacity orjson
import os, json, orjson, time
from dataclasses import dataclass
from typing import Dict, Any, List
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

PROMPT_TMPL = """あなたは日本語ASRの評価者です。出力は必ず有効なJSONのみ。

## 目的
正解テキスト（漢字かな混在）と、各方式の転写結果（reference_mic, throat_small, throat_small_enhanced, throat_large, throat_large_enhanced）を比較し、以下の5段階で意味を最重視して採点します。

## 採点基準（意味を強く反映）
1: 意味が全く合わない（主述/極性/要点が不一致。伝達失敗）
2: 話題は拾えている（トピック近いが要点がズレる/誤情報あり）
3: なんとか意味は通じる（要点は概ね合うが誤り/欠落が目立つ）
4: 表記ゆれ/誤変換はあるが意味は明瞭（情報保持は十分）
5: ほぼ正解と同じ（軽微な表記差のみ）

### 減点の指針
- 重大誤りは大減点：否定/極性の取り違え、数量・固有名詞の誤り、主語述語の取り違え
- 軽微な減点：漢字↔かなの違い、仮名ゆれ、句読点・長音の揺れ、同義語置換
- 同音異義は文脈で意味が変わるなら減点

### 出力形式（JSONのみ）
{{
  "reference_mic": {{"score": <1-5の整数>, "reason_ja": "<40字以内>"}},
  "throat_small": {{"score": <1-5>, "reason_ja": "<40字以内>"}},
  "throat_small_enhanced": {{"score": <1-5>, "reason_ja": "<40字以内>"}},
  "throat_large": {{"score": <1-5>, "reason_ja": "<40字以内>"}},
  "throat_large_enhanced": {{"score": <1-5>, "reason_ja": "<40字以内>"}}
}}

## 入力
[正解テキスト]
{correct_text}

[各方式の転写結果]
- reference_mic: {reference_mic_transcript}
- throat_small: {throat_small_transcript}
- throat_small_enhanced: {throat_small_enhanced_transcript}
- throat_large: {throat_large_transcript}
- throat_large_enhanced: {throat_large_enhanced_transcript}

## 注意
- 出力はJSONのみ。説明文やマークダウンは禁止。
- 判定は意味最優先。表記差は軽減。
- 不明・空欄は「1〜2」を基準に、理由に「欠落」と明記。
"""

@dataclass
class Item:
    # 1発話分のバンドル
    correct_text: str
    reference_mic_transcript: str
    throat_small_transcript: str
    throat_small_enhanced_transcript: str  # ←綴り注意: trascriptではなくtranscript
    throat_large_transcript: str
    throat_large_enhanced_transcript: str
    uid: str  # 識別子（ファイル名等）

def build_prompt(item: Item) -> str:
    # 多少のNoneや空文字に頑健化
    def nz(x): return "" if x is None else str(x)
    return PROMPT_TMPL.format(
        correct_text=nz(item.correct_text),
        reference_mic_transcript=nz(item.reference_mic_transcript),
        throat_small_transcript=nz(item.throat_small_transcript),
        throat_small_enhanced_transcript=nz(item.throat_small_enhanced_transcript),
        throat_large_transcript=nz(item.throat_large_transcript),
        throat_large_enhanced_transcript=nz(item.throat_large_enhanced_transcript),
    )

class JSONParseError(Exception):
    pass

def safe_json_loads(s: str) -> Dict[str, Any]:
    # 余計な前後テキストの除去＆厳格パース
    s = s.strip()
    # JSON以外の行を落とす（最外カッコ抽出）
    first = s.find("{")
    last = s.rfind("}")
    if first == -1 or last == -1:
        raise JSONParseError("No JSON object braced by { } found.")
    core = s[first:last+1]
    try:
        return orjson.loads(core)
    except Exception as e:
        # よくある末尾カンマ等の修正はしない（評価の厳格性を維持）
        raise JSONParseError(str(e))

@retry(
    reraise=True,
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=1, max=8),
    retry=retry_if_exception_type((JSONParseError, Exception))
)
def score_with_gpt(item: Item, model: str = "gpt-5") -> Dict[str, Any]:
    prompt = build_prompt(item)
    resp = client.chat.completions.create(
        model=model,
        #temperature=0.2,
        top_p=1.0,
        messages=[{"role": "user", "content": prompt}],
        # JSON強制モード（対応モデルなら有効）
        response_format={"type": "json_object"}
    )
    text = resp.choices[0].message.content
    data = safe_json_loads(text)
    # 必須キーが無ければ例外（→retry）
    required = [
        "reference_mic","throat_small","throat_small_enhanced",
        "throat_large","throat_large_enhanced"
    ]
    for k in required:
        if k not in data or "score" not in data[k]:
            raise JSONParseError(f"Missing key/score: {k}")
    return data

def flatten_result(uid: str, correct_text: str, result: Dict[str, Any]) -> Dict[str, Any]:
    row = {"uid": uid, "correct_text": correct_text}
    for system in ["reference_mic","throat_small","throat_small_enhanced","throat_large","throat_large_enhanced"]:
        row[f"{system}_score"]  = int(result[system]["score"])
        row[f"{system}_reason"] = result[system].get("reason_ja","")
    return row

def summarize(df: pd.DataFrame) -> pd.DataFrame:
    cols = [c for c in df.columns if c.endswith("_score")]
    mean = df[cols].mean().to_frame("mean_score")
    std  = df[cols].std(ddof=1).to_frame("std")
    cnt  = df[cols].count().to_frame("n")
    summary = mean.join(std).join(cnt)
    summary["rank"] = summary["mean_score"].rank(ascending=False, method="min")
    return summary.sort_values("mean_score", ascending=False)


In [69]:
df.columns

Index(['correct_text', 'reference_mic', 'throat_small',
       'throat_small_enhanced', 'throat_large', 'throat_large_enhanced',
       'reference_mic_transcript', 'throat_small_transcript',
       'throat_small_enhanced_transcript', 'throat_large_transcript',
       'throat_large_enhanced_transcript'],
      dtype='object')

In [71]:
flats = []
for row in df.itertuples():
    item = Item(
        correct_text = row.correct_text,
        reference_mic_transcript = row.reference_mic_transcript,
        throat_small_transcript = row.throat_small_transcript,
        throat_small_enhanced_transcript = row.throat_small_enhanced_transcript,
        throat_large_transcript = row.throat_large_transcript,
        throat_large_enhanced_transcript = row.throat_large_enhanced_transcript,
        uid = f"item_{row.Index+1:04d}"
    )
    result = score_with_gpt(item, model="gpt-5")
    flat = flatten_result(item.uid, item.correct_text, result)
    flats.append(flat)
    #print(flat)
    #break  # 最初の1件だけ試す

In [72]:
df_score = pd.DataFrame(flats)
df_summary = summarize(df_score)

In [73]:
df_score.head()

,uid,correct_text,reference_mic_score,reference_mic_reason,throat_small_score,throat_small_reason,throat_small_enhanced_score,throat_small_enhanced_reason,throat_large_score,throat_large_reason,throat_large_enhanced_score,throat_large_enhanced_reason
0,item_0001,水をマレーシアから買わなくてはならないのです。,5,原文どおりで意味一致,3,対象の水が欠落し頻度も誤り,1,意味不明で主旨不一致、極性も誤り,1,話題逸脱し意味不一致,1,全く別内容で意味が合わない
1,item_0002,木曜日、停戦会談は、何の進展もないまま終了しました。,3,停戦会談が誤変換。要点は辛うじて把握,3,停戦会談が誤変換。要点は辛うじて把握,3,主要語が誤変換、意味は辛うじて通る,2,否定が肯定に転化。話題は近い,1,意味逸脱・固有名詞誤り多数
2,item_0003,上院議員は私がデータをゆがめたと告発した。,5,意味一致、表記差のみ,5,意味一致、表記差のみ,3,主語誤りだが告発内容は概ね合致,1,告発→告白、主語・内容も不一致,1,意味逸脱。主語・述語が大きく不一致
3,item_0004,１週間して、そのニュースは本当になった。,5,表記差のみで意味一致,5,句読点のみ差、意味は完全一致,5,表記差程度で意味は一致,1,時間・対象誤りと否定で不一致,1,期間・主題誤りで意味逸脱
4,item_0005,血圧は、健康のパロメーターとして重要である。,5,語の揺れのみ、意味は一致,5,語の揺れのみ、意味は一致,2,主要語が化けて意味が不明,1,主題逸脱し意味が合わない,1,出力が欠落


In [74]:
df_summary

,mean_score,std,n,rank
reference_mic_score,4.52,0.969015,100,1.0
throat_small_score,2.85,1.395339,100,2.0
throat_large_score,2.25,1.500000,100,3.0
throat_small_enhanced_score,1.53,0.947671,100,4.0
throat_large_enhanced_score,1.38,0.775639,100,5.0


In [75]:
df_with_score =pd.concat([df, df_score], axis=1)

In [76]:
df_with_score.head()

,correct_text,reference_mic,throat_small,throat_small_enhanced,throat_large,throat_large_enhanced,reference_mic_transcript,throat_small_transcript,throat_small_enhanced_transcript,throat_large_transcript,...,reference_mic_score,reference_mic_reason,throat_small_score,throat_small_reason,throat_small_enhanced_score,throat_small_enhanced_reason,throat_large_score,throat_large_reason,throat_large_enhanced_score,throat_large_enhanced_reason
0,水をマレーシアから買わなくてはならないのです。,recordings/Marantz_Umpire_Mic/20251021232813.wav,recordings/External_Microphone/20251021232813.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021232813.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212328...,水をマレーシアから買わなくてはならないのです。,いつもマレーシアから買わなくてはならないのです。,いつも、毎日、やかな顔のってはならないのです。,別でマレーシアからカナダの方への呼びかけです。,...,5,原文どおりで意味一致,3,対象の水が欠落し頻度も誤り,1,意味不明で主旨不一致、極性も誤り,1,話題逸脱し意味不一致,1,全く別内容で意味が合わない
1,木曜日、停戦会談は、何の進展もないまま終了しました。,recordings/Marantz_Umpire_Mic/20251021233140.wav,recordings/External_Microphone/20251021233140.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021233140.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212331...,木曜日、停線階段は何の進展もないまま終了しました。,木曜日、停線階段は何の進展もないまま終了しました。,木曜日、停線改段は何の進展もないまま終了しました。,木曜日、定山会談はこの進展のみを終了しました。,...,3,停戦会談が誤変換。要点は辛うじて把握,3,停戦会談が誤変換。要点は辛うじて把握,3,主要語が誤変換、意味は辛うじて通る,2,否定が肯定に転化。話題は近い,1,意味逸脱・固有名詞誤り多数
2,上院議員は私がデータをゆがめたと告発した。,recordings/Marantz_Umpire_Mic/20251021233208.wav,recordings/External_Microphone/20251021233208.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021233208.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212332...,上院議員は私がデータを歪めたと告発した。,上院議員は私がデータを歪めたと告発した。,教員入院は私がデータを歪めたと告発した。,JNBはあたしのデータを入れ替えたと告白した。,...,5,意味一致、表記差のみ,5,意味一致、表記差のみ,3,主語誤りだが告発内容は概ね合致,1,告発→告白、主語・内容も不一致,1,意味逸脱。主語・述語が大きく不一致
3,１週間して、そのニュースは本当になった。,recordings/Marantz_Umpire_Mic/20251021233250.wav,recordings/External_Microphone/20251021233250.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021233250.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212332...,一週間して、そのニュースは本当になった。,一週間してそのニュースは本当になった,一週間してそのニュースは本当になった。,一生懸命して、そのネオイスは本当にいなかった。,...,5,表記差のみで意味一致,5,句読点のみ差、意味は完全一致,5,表記差程度で意味は一致,1,時間・対象誤りと否定で不一致,1,期間・主題誤りで意味逸脱
4,血圧は、健康のパロメーターとして重要である。,recordings/Marantz_Umpire_Mic/20251021233319.wav,recordings/External_Microphone/20251021233319.wav,recordings/External_Microphone_Enhanced/202510...,recordings/GHW_USB_AUDIO/20251021233319.wav,recordings/GHW_USB_AUDIO_Enhanced/202510212333...,血圧は健康のバロメータとして重要である。,血圧は健康のバロメータとして重要である。,血圧は健康のバロンイーターとして重要である。,お客さんさん、健康のために隠して重要です。,...,5,語の揺れのみ、意味は一致,5,語の揺れのみ、意味は一致,2,主要語が化けて意味が不明,1,主題逸脱し意味が合わない,1,出力が欠落


In [77]:
df_with_score.to_pickle('throat2air_sklearn_result.pkl')